In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd

coor_data = pd.read_csv('Data/coordata.csv')
date_data = pd.read_csv('Data/datedata.csv')
meta_data = pd.read_csv('Data/metadata.csv')
bus_stop_data = pd.read_csv('Data/bus_stop.csv')

# general = pd.read_csv('Data/ScooterData_July15_Sept27_2019.csv')

data_set = coor_data.join(bus_stop_data.join(date_data.join(meta_data)))


Y_data = data_set[['endx', 'endy']].to_numpy()
X_data = data_set.drop(columns=['endx', 'endy','start_date', 'end_near_stop']).to_numpy()

I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
I:\Users\PC\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework

In [2]:
import progressbar

X_sets_by_day = []
Y_sets_by_day = []

hr = 0
x = []
y = []

compare_hr_before = 0
compare_hr_current = 0
for i in progressbar.progressbar(range(len(X_data))):
    compare_hr_current = data_set['start_hour'][i]
    if (compare_hr_before == 23 and compare_hr_current == 0):
        hr = 0
        X_sets_by_day.append(x)
        Y_sets_by_day.append(y)
        x = []
        y = []
    compare_hr_before = data_set['start_hour'][i]
    x.append(X_data[i])
    y.append(Y_data[i])
    if i == (len(X_data) - 1):
        X_sets_by_day.append(x)
        Y_sets_by_day.append(y)

100% (459478 of 459478) |################| Elapsed Time: 0:00:07 Time:  0:00:07


In [63]:
X_train_set = []
Y_train_set = []
for i in progressbar.progressbar(range(48)):
    for j in range(len(X_sets_by_day[i])):
        X_train_set.append(X_sets_by_day[i][j])
        Y_train_set.append(Y_sets_by_day[i][j])

X_test_set = []
Y_test_set = []
for i in progressbar.progressbar(range(48,len(X_sets_by_day))):
    for j in range(len(X_sets_by_day[i])):
        X_test_set.append(X_sets_by_day[i][j])
        Y_test_set.append(Y_sets_by_day[i][j])
        
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.callbacks import EarlyStopping

X_train_set = np.array(X_train_set)
X_test_set = np.array(X_test_set)
scx = MinMaxScaler(feature_range=(0,1))
X_train_set_fit = scx.fit_transform(X_train_set)
X_test_set_fit = scx.transform(X_test_set)

Y_train_set = np.array(Y_train_set)
Y_test_set = np.array(Y_test_set)
scy = MinMaxScaler(feature_range=(0,1))
Y_train_set_fit = scy.fit_transform(Y_train_set)
Y_test_set_fit = scy.transform(Y_test_set)

100% (48 of 48) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (27 of 27) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [64]:
from tensorflow.keras import layers, Model, regularizers
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow.python.keras.layers.recurrent import LSTM
from copy import copy



X_train_set_by_day = []
Y_train_set_by_day = []
X_test_set_by_day = []
Y_test_set_by_day = []

inputA = layers.Input(shape=(4))
inputB = layers.Input(shape=(60))

x = Dense(15)(inputA)
x = Dense(30, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
y = Dense(62)(inputB)
y = Dense(70, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)

combined = layers.concatenate([x.output, y.output])

final = Dense(70, activation="softmax")(combined)
a = Dense(2)(final)

# X_train_set_by_day.append(X_train_set_fit[X_train_set[:,4] == day])
# Y_train_set_by_day.append(Y_train_set_fit[X_train_set[:,4] == day])
# X_test_set_by_day.append(X_test_set_fit[X_test_set[:,4] == day])
# Y_test_set_by_day.append(Y_test_set_fit[X_test_set[:,4] == day])
model = Model(inputs=[x.input,y.input], outputs=a)
model.compile(loss="mean_absolute_error", optimizer="adam")
# X_train_set_by_day.append(X_train_set_fit[X_train_set[:,4] < 5])
# Y_train_set_by_day.append(Y_train_set_fit[X_train_set[:,4] < 5])
# X_test_set_by_day.append(X_test_set_fit[X_test_set[:,4] < 5])
# Y_test_set_by_day.append(Y_test_set_fit[X_test_set[:,4] < 5])
# models.append(Model(inputs=[x.input,y.input], outputs=a))
# models[0].compile(loss="mean_absolute_error", optimizer="adam")

# X_train_set_by_day.append(X_train_set_fit[X_train_set[:,4] >= 5])
# Y_train_set_by_day.append(Y_train_set_fit[X_train_set[:,4] >= 5])
# X_test_set_by_day.append(X_test_set_fit[X_test_set[:,4] >= 5])
# Y_test_set_by_day.append(Y_test_set_fit[X_test_set[:,4] >= 5])
# models.append(Model(inputs=[x.input,y.input], outputs=a))
# models[1].compile(loss="mean_absolute_error", optimizer="adam")

In [65]:
def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 5)
    km = round(km, 5)
    return km

In [ ]:

x_train_old = []
y_train_old = []


x_train = X_train_set_fit
y_train = Y_train_set_fit


model.fit([x_train[:,0:4],x_train[:,4:]], y_train, epochs=50, verbose=2)  

mae_rate_day = np.zeros(7)
mae_count_day = np.zeros(7)
mae_rate_hour = np.zeros(24)
mae_count_hour = np.zeros(24)
error_day_sum = []
for i in range(48,75):
    print('current at: '+str(i))
    x = scx.transform(X_sets_by_day[i])
    y = scy.transform(Y_sets_by_day[i])
    X_train_set_fit = np.concatenate((X_train_set_fit, x), axis=0)
    Y_train_set_fit = np.concatenate((Y_train_set_fit, y), axis=0)
    x_train = X_train_set_fit
    y_train = Y_train_set_fit

    ss = models[day].evaluate([x[:,0:4],x[:,4:]], y)

    result_Y = models[day].predict([x[:,0:4],x[:,4:]])
    result_Y_all = scy.inverse_transform(result_Y)
    y_test_all = scy.inverse_transform(y)

    error_all = 0            
    x_tr = scx.inverse_transform(x)
#         for i in (range(len(y_test_all))):
#             error_all = error_all + haversine(result_Y_all[i],y_test_all[i])
#         error_avg = error_all/len(y_test_all)
#         print(round(error_avg, 3))
#         mae_rate_day[day] = mae_rate_day[day] + error_avg
    for dr in range(7):
        mask = np.logical_and(x_tr[:,4] > (dr)-0.2, x_tr[:,4] < (dr+1)-0.2)
        result_Y_d = result_Y_all[mask]
        if len(result_Y_d) > 0:
            Y_test_in = y_test_all[mask]
            error_all = 0
            for i in (range(len(Y_test_in))):
                error_all = error_all + haversine(result_Y_d[i],Y_test_in[i])
            error_avg = error_all/len(Y_test_in)
            mae_rate_day[day] = mae_rate_day[day] + error_avg
            mae_count_day[day] = mae_count_day[day] + 1

    for hr in range(24):
        mask = np.logical_and(x_tr[:,3] > (hr)-0.2, x_tr[:,3] < (hr+1)-0.2)
        result_Y_d = result_Y_all[mask]
        Y_test_in = y_test_all[mask]
        error_all = 0
        for i in (range(len(Y_test_in))):
            error_all = error_all + haversine(result_Y_d[i],Y_test_in[i])
        error_avg = error_all/len(Y_test_in)
        mae_rate_hour[hr] = mae_rate_hour[hr] + error_avg
        mae_count_hour[hr] = mae_count_hour[hr] + 1

    history = models[day].fit([x_train[:,0:4],x_train[:,4:]], y_train, epochs=50, verbose=2)

mae_rate_day[day] = mae_rate_day[day]/mae_count_day[0]
mae_rate_hour = mae_rate_hour/mae_count_hour[0]

Epoch 1/50
272446/272446 - 18s - loss: 0.0295
Epoch 2/50
272446/272446 - 20s - loss: 0.0189
Epoch 3/50
272446/272446 - 20s - loss: 0.0187
Epoch 4/50
272446/272446 - 20s - loss: 0.0186
Epoch 5/50
272446/272446 - 19s - loss: 0.0185
Epoch 6/50
272446/272446 - 20s - loss: 0.0185
Epoch 7/50
272446/272446 - 20s - loss: 0.0185
Epoch 8/50
272446/272446 - 20s - loss: 0.0184
Epoch 9/50
272446/272446 - 19s - loss: 0.0184
Epoch 10/50
272446/272446 - 14s - loss: 0.0184
Epoch 11/50
272446/272446 - 18s - loss: 0.0184
Epoch 12/50
272446/272446 - 19s - loss: 0.0184
Epoch 13/50
272446/272446 - 16s - loss: 0.0184
Epoch 14/50
272446/272446 - 16s - loss: 0.0183
Epoch 15/50
272446/272446 - 13s - loss: 0.0183
Epoch 16/50
272446/272446 - 14s - loss: 0.0183
Epoch 17/50
272446/272446 - 13s - loss: 0.0183
Epoch 18/50
272446/272446 - 19s - loss: 0.0183
Epoch 19/50
272446/272446 - 21s - loss: 0.0183
Epoch 20/50
272446/272446 - 20s - loss: 0.0183
Epoch 21/50
272446/272446 - 20s - loss: 0.0183
Epoch 22/50
272446/272

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
result_Y = model.predict([X_test_set_fit[:,0:4],X_test_set_fit[:,4:]])

result_Y_plot = scy.inverse_transform(result_Y)
y_test_plot = scy.inverse_transform(Y_test_set_fit)

plt.scatter((y_test_plot[:,0]),(y_test_plot[:,1]), color='b', s= 1)
plt.scatter((result_Y_plot[:,0]), (result_Y_plot[:,1]), color='r', s= 1)



In [32]:
mae_rate_hour = []

x_test_hour = scx.inverse_transform(X_test_set_fit)

result_Y_hour = scy.inverse_transform(result_Y)
y_test_hour = scy.inverse_transform(Y_test_set_fit)
for index in range(24):
    mask = np.logical_and(x_test_hour[:,3] > (index)-0.2, x_test_hour[:,3] < (index+1)-0.2)
    result_Y_d = result_Y_hour[mask]
    Y_test_in = y_test_hour[mask]
    error = 0
    for i in range(len(Y_test_in)):
        error = error + haversine(result_Y_d[i,0:2],Y_test_in[i,0:2])
    end = error/len(Y_test_in)
    error_average_distance = end

    y_test[:,0] = y_test[:,0]
    mae_rate_hour.append([float(error_average_distance), str(index)])

mae_rate_hour = np.array(mae_rate_hour, dtype=np.object)
plt.xlabel('hours')
plt.ylabel('error_avg_distance(km)')
plt.grid(color='black', linestyle='-', linewidth=0.3)
plt.scatter(mae_rate_hour[:,1], mae_rate_hour[:,0], color='r', s= 1)


NameError: name 'y_test' is not defined

In [12]:
sum(scat_plot)/len(scat_plot)

NameError: name 'scat_plot' is not defined

In [ ]:
import matplotlib.pyplot as plt

mae_rate_day_of_week = []

x_test_day = scx.inverse_transform(X_test_set_fit)

result_Y_day = scy.inverse_transform(result_Y)
y_test_day = scy.inverse_transform(Y_test_set_fit)
for index in range(7):
    mask = np.logical_and(x_test_day[:,4] > (index)-0.1, x_test_day[:,4] < (index+1)-0.1)
    result_Y_d_day = result_Y_day[mask]
    Y_test_in = y_test_day[mask]
    error = 0
    haverList = []
    for i in (range(len(Y_test_in))):
        error = error + haversine(result_Y_d_day[i,0:2],Y_test_in[i,0:2])
    end = error/len(Y_test_in)
    error_average_distance = end

    mae_rate_day_of_week.append([error_average_distance, str(index+1)])

mae_rate_day_of_week = np.array(mae_rate_day_of_week, dtype=np.object)
plt.xlabel('days of week')
plt.ylabel('error_avg_distance(km)')
plt.grid(color='black', linestyle='-', linewidth=0.3)
plt.scatter(mae_rate_day_of_week[:,1], mae_rate_day_of_week[:,0], color='r', s= 1)